In [15]:
import os
import ollama
from anytree import PreOrderIter
import json
from naive_baseline import generate_naive_baseline
from utils import generate_formal_explanations, make_decision
import pandas as pd

In [16]:
model_name = "llama3" # "mistral"

### Changed pipeline a bit, added preorder_traversal list to follow the order in which the agent decides what to do;
### function generate_comprehensive_explanation generates response from mistral https://ollama.com/library/mistral

In [17]:
def preorder_traversal(root):
    """
    Performs a preorder traversal of a tree and returns a list of node names.
    
    :param root: The root node of the tree.
    :return: A list of node names in preorder traversal order.
    """
    preorder_traversal_nodes = []
    
    for node in PreOrderIter(root):
        current_node_name = node.name
        preorder_traversal_nodes.append(current_node_name)
    
    return preorder_traversal_nodes

def generate_natural_language_explanation(json_tree, naive_explanation, formal_explanations, tree_traversal_order, 
                                     norm, beliefs, goal, preferences, action_to_explain, chosen_trace=None, bad_example=""):
    
    # Format key context elements in plain English
    norm_str = norm if norm else "None"
    beliefs_str = ", ".join(beliefs) if beliefs else "None"
    goal_str = goal if goal else "None"
    pref_description = f"{preferences[0]} with priority {preferences[1]}"
    chosen_trace_str = ", ".join(chosen_trace) if chosen_trace else "None"

    # prompt = f"""
    # You are an expert in explaining the reasoning behind actions taken by an agent in a decision-making process. Your task is to provide a natural language explanation of the reasoning behind the agent's actions, specifically focusing on the action '{action_to_explain}' and the path it took to reach its final decision.
    # You will be given naive_explanation that describe the agent's reasoning process in a simple way, and you need to BETTER PHARSE THEM but NOT ADD ANYTHING NEW.
    # Take into acount the Tree traversal order the agent took to generate its naive explanation, maitain the same order in your explanation.
    # You will also be provided with a set of beliefs, norms, restrictions, and goals that the agent had at the beginning of the process. Your explanation should be comprehensive and cover all aspects of the reasoning process, including the initial beliefs, norms, restrictions, goals, and preferences that influenced the agent's decisions start with all initial info that agent got and then dive into the path he walked alonge the tree.

    # ### CORE DATA (reference only)
    # - Action to explain: '{action_to_explain}' - the action that needs to be explained (can either be in the chosen path or not)
    # - My chosen path: {chosen_trace_str} - the selected actions that were chosen at the end
    # - Initial beliefs: {beliefs_str} - the set of beliefs that the agent starts with
    # - Norms/restrictions: {norm_str} - the set of actionsY that are either prohibited or obligatory. P means prohibited, O means obligatory.
    # - Goal: {goal_str} - the set of beliefs that must be true at the end of the execution of the trace
    # - Preferences: {pref_description} - given they by list of factors and then their order of importance
    # - Tree traversal: {tree_traversal_order} - the order in which nodes were visited in the tree to generate formal explanations
    # - naive_explanation: {naive_explanation} - the natural language explanation generated using the naive baseline approach
    # - bad_example: {bad_example} - a bad example of a natural language explanation that should be avoided


    # ### CRITICAL INSTRUCTIONS
    # First, determine if '{action_to_explain}' is in your path {chosen_trace_str}, but DO NOT include this verification in your explanation.

    # ABSOLUTELY DO NOT USE:
    # - Section headings (like "Initial Conditions:" or "Reasoning Process:")
    # - Debugging statements (like "I verify that...")
    # - Numbered or bulleted lists
    # - Technical phrases (like "natural language translation")
    # - Any meta-text about the explanation structure
    # - Learn from the bad example provided and avoid similar mistakes, see suggestions and notes inside the bad example

    # Your explanation MUST seamlessly integrate ALL of the following:

    # 1. A clear statement of whether you performed '{action_to_explain}' or not
    # 2. A description of the complete path you took in everyday language
    # 3. A thorough explanation of ALL initial beliefs, goals, norms, restrictions and preferences. DO NOT list them - explain them naturally and DO NOT miss anything.
    # Especially, explain how the norms and preferences influenced your decisions e.g. if you could not perform an action because it was prohibited.
    # 4. A detailed account of each decision point where you:
    # - Describe ALL options you considered at each step
    # - Explain specifically why some options were viable and others weren't
    # - Connect your choices directly to your initial beliefs and preferences
    # - Show how each decision led to the next one
    # - If the action is not in the path, explain why it was not chosen, especially if it was not a valid one because of received norms or restrictions
    
    # Remember: While being conversational, you must still be comprehensive - every belief, preference, option considered, and reasoning step needs to be included, just expressed naturally.
    # """

    prompt = f"""
    I am an agent making decisions based on my beliefs, goals, and restrictions. Your task is to **rewrite my naive explanation 
    into a natural, first-person account** while keeping all reasoning intact.

    ### CONTEXT
    - **Action to explain**: '{action_to_explain}'  
    - **Chosen path**: {chosen_trace_str}  
    - **Initial beliefs**: {beliefs_str}  
    - **Norms/restrictions**: {norm_str}  
    - **Goal**: {goal_str}  
    - **Preferences**: {pref_description}  
    - **Tree traversal order**: {tree_traversal_order}  
    - **Naive explanation**: {naive_explanation}  

    ### INSTRUCTIONS
    - Rewrite the explanation in **first-person**, as if I (the agent) am telling the story of my decision-making. 
    - Start your answer natrually, as if you are explaining to a friend. 
    - Assume you start at 'getCoffee' root node so you don't need to explain it.
    - Follow the same **reasoning order** as my naive explanation.  
    - Start by explaining my **beliefs, goals, and restrictions naturally** (avoid lists).  
    - Note that a lower cost for action attribute means a better option so pharse it in a way that the reader understands that.
    - Walk through my **thought process step by step**, describing options I considered, why I rejected some, and how I made my choices.  
    - If '{action_to_explain}' was **not chosen**, explain why in a way that feels natural and clear.  
    - Keep it **conversational and engaging**, avoiding generic or robotic phrasing.  

    **DO NOT:**  
    - Use section headings or bullet points.  
    - State information in a dry, factual way without making it flow naturally.  
    - Add anything new—just improve clarity and expressiveness.  
    - Say you didn't do a step you actually did.

    I want my explanation to sound like **I'm actually explaining my reasoning to someone** rather than listing facts.
    """
    
    # Send the prompt to your LLM
    messages = [
        {
            'role': 'user',
            'content': prompt
        }
    ]
    
    # response = requests.post(f"{api_base}/chat", json={"model": "mistral", "messages": messages})
    
    response = ollama.chat(model=model_name, messages=messages)
    
    try:
        return response['message']['content']
    except Exception as e:
        print(f"JSONDecodeError: {e}")
        return None

#### Integrated here llm explanations and save them into explanations/mistral

In [18]:
def generate_explanations_for_inputs(test_cases, cases_to_do):
    """
    Generate explanations for the inputs test cases
    :param test_cases: dictionary containing the test cases
    :param cases_to_do: list of test cases to actually generate explanations for (for easy re-running)
    :return: dictionary containing the explanations for each test case
    """
    
    # in .ipynb current directory is extracted like this
    current_dir = os.getcwd()

    # current_dir = os.path.dirname(__file__)

    baseline_explanations_dir = os.path.join(current_dir, "explanations", "baseline")
    os.makedirs(baseline_explanations_dir, exist_ok=True)


    llm_explanations_dir = os.path.join(current_dir, "explanations", model_name)
    
    baseline_explanations = {}
    llm_explanations = {}
    for index, row in test_cases.iterrows():    
        if index not in cases_to_do:
            continue 
        print(f"Processing test case {index}")   
        # Read the JSON file into a dictionary
        with open(f'{current_dir}/{row["name_json_tree_file"]}', 'r') as file:
            json_tree = json.load(file)
        # Generate formal explanations
        formal_explentions, chosen_trace = generate_formal_explanations(json_tree=json_tree, norm=row["norm"], beliefs=row["beliefs"], goal=row["goal"], preferences=row["preferences"], action_to_explain=row["action_to_explain"])
        baseline_explanations[index] = generate_naive_baseline(formal_explentions=formal_explentions, chosen_trace=chosen_trace, norm=row["norm"], beliefs=row["beliefs"], goal=row["goal"], preferences=row["preferences"], action_to_explain=row["action_to_explain"])



        root, chosen_trace, valid_traces, valid_costs = make_decision(json_tree, norm=row["norm"], goal=row["goal"], beliefs=row["beliefs"], preferences=row["preferences"], output_dir='/')
        tree_traversal_order = preorder_traversal(root=root)

        # read bad example file
        bad_example_file = os.path.join(current_dir, "explanations", "bad", f"{index}.txt")
        bad_example = ""
        with open(bad_example_file, 'r') as file:
            bad_example = file.read()


        llm_explanations[index] = generate_natural_language_explanation(
                        json_tree=json_tree,
                        naive_explanation=baseline_explanations[index],
                        formal_explanations=formal_explentions,
                        tree_traversal_order=tree_traversal_order,
                        norm=row["norm"],
                        beliefs=row["beliefs"],
                        goal=row["goal"],
                        preferences=row["preferences"],
                        action_to_explain=row["action_to_explain"],
                        chosen_trace=chosen_trace,
                        bad_example=bad_example
                        )

        # Save each explanation to a separate text file
        baseline_explanation_file = os.path.join(baseline_explanations_dir, f"{index}.txt")
        with open(baseline_explanation_file, 'w') as f:
            for explanation in baseline_explanations[index]:
                f.write(explanation + '\n')


        os.makedirs(llm_explanations_dir, exist_ok=True)  

        llm_explanation_file = os.path.join(llm_explanations_dir, f"{index}.txt")

        print(f"directory ---> {llm_explanation_file}")
        
        # Loop through the dictionary and write explanations for each test case
        with open(llm_explanation_file, 'w') as f:
            for explanation in llm_explanations[index]:
                # Write the explanation for this test case
                f.write(explanation)   # Add an extra newline for separation

        
    
    return baseline_explanations, llm_explanations

In [19]:
test_cases = [
    {
        "name_json_tree_file": "coffee.json",
        "norm": {"type": "P", "actions": ["payShop"]},
        "beliefs": ["staffCardAvailable", "ownCard", "colleagueAvailable", "haveMoney", "AnnInOffice"],
        "goal": ["haveCoffee"],
        "preferences": [["quality", "price", "time"], [1, 2, 0]],
        "action_to_explain": "payShop"
    },
    {
        "name_json_tree_file": "coffee.json",
        "norm": {},
        "beliefs": ["staffCardAvailable", "ownCard"],
        "goal": ["haveCoffee"],
        "preferences": [["quality", "price", "time"], [0, 1, 2]],
        "action_to_explain": "getCoffeeKitchen"
    },
    {
        "name_json_tree_file": "coffee.json",
        "norm": {},
        "beliefs": ["haveMoney", "AnnInOffice"],
        "goal": ["haveCoffee"],
        "preferences": [["quality", "price", "time"], [0, 1, 2]],
        "action_to_explain": "getCoffeeShop"
    },
    {
        "name_json_tree_file": "coffee.json",
        "norm": {"type": "P", "actions": ["gotoAnnOffice"]},
        "beliefs": ["staffCardAvailable", "ownCard", "colleagueAvailable", "haveMoney", "AnnInOffice"],
        "goal": ["haveCoffee"],
        "preferences": [["quality", "price", "time"], [0, 1, 2]],
        "action_to_explain": "payShop"
    },
    {
        "name_json_tree_file": "coffee.json",
        "norm": {"type": "P", "actions": ["payShop"]},
        "beliefs": ["staffCardAvailable", "ownCard", "colleagueAvailable", "haveMoney"],
        "goal": ["haveCoffee"],
        "preferences": [["quality", "price", "time"], [1, 2, 0]],
        "action_to_explain": "gotoKitchen"
    }
]

df_test_cases = pd.DataFrame(test_cases, index=[f"test_case_{i+1}" for i in range(len(test_cases))])

In [ ]:
cases_to_do = ["test_case_1", "test_case_2"]#, "test_case_3", "test_case_4", "test_case_5"]
baseline_explanations, llm_explanations = generate_explanations_for_inputs(df_test_cases, cases_to_do)

Processing test case test_case_1
Chosen trace: ['getCoffee', 'getKitchenCoffee', 'getStaffCard', 'getOwnCard', 'gotoKitchen', 'getCoffeeKitchen']
Action to explain: payShop not in trace
directory ---> c:\Users\bar24\OneDrive - Universiteit Utrecht\Documents\School\UU Data Sceince MSc\1st Year\Period 3\Explainable AI - INFOMXAI\Assignments\Project2\ExplainableAI-Project2\Part2\explanations\llama3\test_case_1.txt
Processing test case test_case_2
Chosen trace: ['getCoffee', 'getKitchenCoffee', 'getStaffCard', 'getOwnCard', 'gotoKitchen', 'getCoffeeKitchen']
Action to explain: getCoffeeKitchen in trace
Ancestor names: ['getCoffee', 'getKitchenCoffee']
directory ---> c:\Users\bar24\OneDrive - Universiteit Utrecht\Documents\School\UU Data Sceince MSc\1st Year\Period 3\Explainable AI - INFOMXAI\Assignments\Project2\ExplainableAI-Project2\Part2\explanations\llama3\test_case_2.txt
Processing test case test_case_3
Chosen trace: ['getCoffee', 'getShopCoffee', 'gotoShop', 'payShop', 'getCoffeeShop

In [21]:
print("Done!")

Done!
